In [1]:
from flask import Flask, render_template_string, request
import sounddevice as sd
from scipy.io.wavfile import write
import speech_recognition as sr
import pyttsx3
from werkzeug.serving import run_simple
from datetime import datetime


In [2]:
app = Flask(__name__)
recognizer = sr.Recognizer()


In [3]:
def record_audio(duration=5, samplerate=16000):
    print("Recording started...")
    audio_data = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16')
    sd.wait()  # Wait until recording is finished
    write("temp_audio.wav", samplerate, audio_data)
    print("Recording finished and saved as temp_audio.wav")


In [4]:
def generate_response(user_input):
    user_input = user_input.lower()
    if "hello" in user_input:
        return "Hello! I am Dr. Emmanuel Joy! How can I assist you today?"
    elif "time" in user_input:
        return f"The current time is {datetime.now().strftime('%H:%M:%S')}."
    elif "bye" in user_input:
        return "Goodbye! Have a great day. See you in AI ML."
    elif "project" in user_input:
        return "How can I guide you in your project?"
    elif "ai" in user_input:
        return "Artificial Intelligence is transforming the world. So glad you are in my class."
    else:
        return "I'm not sure how to respond to that."


In [5]:
def speak(response):
    engine = pyttsx3.init()
    engine.say(response)
    engine.runAndWait()


In [6]:
@app.route("/")
def home():
    html_content = """
    <!DOCTYPE html>
    <html>
    <head><title>Voice Bot</title></head>
    <body>
        <h1>Voice Bot</h1>
        <form action="/get_response" method="post">
            <button type="submit">Speak</button>
        </form>
    </body>
    </html>
    """
    return render_template_string(html_content)


In [7]:
@app.route("/get_response", methods=["POST"])
def get_response():
    try:
        record_audio(duration=5)  # Record audio for 5 seconds

        with sr.AudioFile("temp_audio.wav") as source:
            audio = recognizer.record(source)
            user_input = recognizer.recognize_google(audio)  # Convert speech to text

        response = generate_response(user_input)  # Generate bot response
        speak(response)  # Convert response text to speech

        return f"User said: {user_input}<br>Bot Response: {response}"

    except Exception as e:
        return f"Error: {e}"


In [ ]:
if __name__ == "__main__":
    run_simple("localhost", 5000, app)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Oct/2025 06:29:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 06:29:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2025 06:29:40] "POST /get_response HTTP/1.1" 200 -


Recording started...
